本代码使用kaggle notebook运行时

训练部分使用TPU

推断部分使用GPU

# 数据添加
https://www.kaggle.com/meminozturk/efficientnetrepo110

https://www.kaggle.com/yueqiangqin/kerasapplications

https://www.kaggle.com/ragnar123/landmark-image-train

https://www.kaggle.com/ragnar123/landmark-tfrecords-384

https://www.kaggle.com/ragnar123/landmark-tfrecords-384-2

https://www.kaggle.com/c/landmark-recognition-2021

# Google Landmark Recognition 2021
随着图像检索和实例识别技术的迅速发展，急需有效的基准数据来对不断出现算法的性能进行有效测评。在这场比赛中，我们通过改进特征提取以及检索算法，给定一张图像，需要在指定图像库识别出所属地标标签并给予评分。

所有图像数据是通过网络数据挖掘而来，来自世界各地的景观拍摄，也有部分图片属于非景点，比如人物，车，树木等等；因此数据集非常庞大且含噪声。

Google Landmarks Dataset v2(GLD2)含近500万幅图像，训练集4132914幅图像，203094个类别，索引集761757幅图像，测试集117577幅图像，该数据集是由网络数据挖掘而得到的地标图像数据，所以含有非常多的噪声标注。

另外该数据集有一个被往届比赛中smlyaka团队使用自动清理工具清理过的版本CGLD2，其训练集含有 1580470 幅图像， 81313 个类别。

## Evaluation 评价指标
### GAP(Global Average Precision全局平均精度)

<img src="https://latex.codecogs.com/gif.latex?GAP&space;=&space;\frac{1}{M}\sum_{i=1}^N&space;P(i)&space;rel(i)" title="GAP = \frac{1}{M}\sum_{i=1}^N P(i) rel(i)" />

真实标签
```
    id,ground_truth
    000088da12d664db,11807
    0001623c6d808702,2188
    0001bbb682d45002,1054
    000270c9100de789,12801
    0002b0fab5d3ccc4,NoLandmark
```
预测值
```
    id,landmarks
    000088da12d664db,11807 0.09
    0001623c6d808702,2188 0.13
    0001bbb682d45002,7933 0.01
    000270c9100de789,12801 0.38
    0002b0fab5d3ccc4,6560 0.19
```
```
重排：
ranked_predictions = [(12801, 0.38), (6560, 0.19), (2188, 0.13), (11807, 0.09), (7933, 0.01)]
确定评价图片数量：
N = len(ranked_predictions)
M = N - 1 #有一张图片没有标签值
计算单个精度：
P(1)=1, P(3)=2/3, P(4)=3/4
求总值：
GAP = 1/M * (P(1) * 1 + P(2) * 0 + P(3) + 1 + P(4) * 1 + P(5) * 0)

```

## Loss目标函数
### Embeddings, Cosine Distance, and ArcFace

### Classification CNN
普通的图像分类模型一般包括三个模块：输入，中间层，输出。输入就是处理过的图片数据，中间层属于特征提取，输出就是预测分类值。

正常来说，我们不需要关注embedding(嵌入向量)，因为一般通过输入图片我们训练一个CNN分类模型，然后得到代表图像类别的one-hot(独热编码)向量。就好比下图，我们训练一个动物分类模型，输入一张图片，然后预测该图片属于动物类别4。

![CNN](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/Mar-2021/arcface.png)

### Embeddings
假设现在需要比较两张图片，确认这两张图片是否相似。从机器角度，图片是难以比较的，但是将图片化为数值，那么就简单多了。因此，我们输入一张图片到模型，提取输出层前一层的输出值，这就是图片的嵌入向量。按照上面的模型图，输入需要比较的两张图片，得到两个128维度的表征向量，然后比较这两个向量。

### Cosine Distance
一般比较两个向量我们可以用计算距离，假如两个向量分别为[0.2, 0.9, 0.7] 、 [0.5, 0.4, 0.1]，那么我们就可以用Euclidean distance(欧几里得距离)计算：
``` 
    sqrt( (0.5-0.2)**2 + (0.4-0.9)**2 + (0.1-0.7)**2 )
```
余弦相似度，就是计算两个向量间的夹角的余弦值；余弦距离就是用1减去这个获得的余弦相似度，余弦距离的取值范围为[0,2]，数值越大，表明图像越相似。

### ArcFace
首先从通用的角度来考虑深度特征学习任务，包括人脸识别、行人再识别、细粒度图像检索等。当给定一个查询图像时，无论图像内容是什么，都希望在库图像中找到一张最相似的候选图像与之匹配。
这些特征学习任务具有两个关键要素，分别是特征的类内紧凑度和类间分离度。在特征学习任务中，我们希望优化目标能够最大化类内相似性Sp，同时最小化类间相似性Sn。深度特征学习有两种经典的学习范式。当拥有类别标签时，可以通过使用分类器和分类损失函数来进行训练，从而学习到一个深度特征空间。经典的softmax函数为了向加强类内紧凑度和类间分离度两个方向发展，逐渐演变成ArcFcae,相似的还有CosFace等。


![softmax](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/Mar-2021/mnist_vgg8_3d.png)

![arcface](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/Mar-2021/mnist_vgg8_arcface_3d.png)

论文链接：
https://arxiv.org/pdf/1801.07698.pdf

In [ ]:
import operator
import gc
import pathlib
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from scipy import spatial
import cv2
!pip install ../input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/efficientnetrepo110/efficientnet-1.1.0-py3-none-any.whl
import efficientnet.tfkeras as efn
import math

NUMBER_OF_CLASSES = 81313
IMAGE_SIZE = [384, 384]
LR = 0.0001

class ArcMarginProduct(tf.keras.layers.Layer):

    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

## Data数据处理(若已生成，可跳过)
利用TensorFlow的TFRecord格式，将大量的图片压缩并上传到kaggle数据集系统。

* 可以压缩图片存储空间
* 便于使用TPU训练
* 训练时加快读取数据速度



https://www.tensorflow.org/tutorials/load_data/tfrecord

In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import pathlib
from tqdm import tqdm

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(feature0, feature1, feature2):
    feature = {
        'id': _bytes_feature(feature0),
        'image': _bytes_feature(feature1),
        'target': _int64_feature(feature2)
    }
    example_proto = tf.train.Example(features = tf.train.Features(feature = feature))
    return example_proto.SerializeToString()
TRAIN_IMAGE_DIR = '../input/landmark-recognition-2021/train'
TRAIN = '../input/landmark-image-train/train_encoded.csv'

# Read image and resize it
def read_image(image_path, size = (384, 384)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    return img


def get_tf_records(record = 0, size = (384, 384)):
    df = pd.read_csv(TRAIN)
    # Get image paths
    image_paths = [x for x in pathlib.Path(TRAIN_IMAGE_DIR).rglob('*.jpg')]
    # Get only one group, this is a slow process so we need to make 50 different sessions
    df = df[df['group'] == record]
    # Reset index 
    df.reset_index(drop = True, inplace = True)
    # Get a list of ids
    ids_list = list(df['id'].unique())
    # Write tf records
    with tf.io.TFRecordWriter('train_{}.tfrec'.format(record)) as writer:
        for image_path in tqdm(image_paths):
            image_id = image_path.name.split('.')[0]
            if image_id in ids_list:
                # Get target
                target = df[df['id'] == image_id]['landmark_id_encode']
                img = read_image(str(image_path), size)
                img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tostring()
                example = serialize_example(
                    str.encode(image_id), img, target.values[0]
                )
                writer.write(example)
                
#get_tf_records(record = 0, size = (384, 384))

## Model模型构建
1. 建立数据读取流水线
2. 模型构建
3. 训练

In [ ]:
import os
#os.system('pip install -q tensorflow~=2.2.0 tensorflow_gcs_config~=2.2.0')
os.system('pip install -q efficientnet')
#os.system('pip install -q gcsfs')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import re
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from tqdm.notebook import tqdm as tqdm
import requests
from kaggle_datasets import KaggleDatasets
#resp = requests.post("http://{}:8475/requestversion/{}".format(os.environ["COLAB_TPU_ADDR"].split(":")[0], tf.__version__))
#if resp.status_code != 200:
#    print("Failed to switch the TPU to TF {}".format(version))

try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

#
# Data access
GCS_PATH = KaggleDatasets().get_gcs_path('landmark-tfrecords-384')#'gs://kds-3873db5988cab73fb3f61387f4260e23cbff4012248d13a9fa5dd335'
#KaggleDatasets().get_gcs_path('landmark-tfrecords-384')
GCS_PATH_2 = KaggleDatasets().get_gcs_path('landmark-tfrecords-384-2')#'gs://kds-e44a20ea6436cd0271b97c5b431450bcd70a463206a983e1acb88874'
#KaggleDatasets().get_gcs_path('landmark-tfrecords-384-2')
DICT_PATH = KaggleDatasets().get_gcs_path('landmark-image-train')+'/train_encoded.csv'
#KaggleDatasets().get_gcs_path('landmark-image-train')

# Configuration
EPOCHS = 20
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [384, 384]
# Seed
SEED = 100
# Learning rate
LR = 0.0001
# Number of classes
NUMBER_OF_CLASSES = 81313
VALID_STAGE = False
# Training filenames directory
files1 = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
files2 = tf.io.gfile.glob(GCS_PATH_2 + '/train*.tfrec')
assert len(files1)>0
assert len(files2)>0
FILENAMES = files1 + files2
# Read csv file
df = pd.read_csv(DICT_PATH)
# Using 20% of the data to validate
TRAINING_FILENAMES, VALIDATION_FILENAMES = train_test_split(FILENAMES, test_size = 0.10, random_state = SEED)
training_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in TRAINING_FILENAMES]
validation_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in VALIDATION_FILENAMES]
n_trn_classes = df[df['group'].isin(training_groups)]['landmark_id_encode'].nunique()
n_val_classes = df[df['group'].isin(validation_groups)]['landmark_id_encode'].nunique()
print(f'The number of unique training classes is {n_trn_classes} of {NUMBER_OF_CLASSES} total classes')
print(f'The number of unique validation classes is {n_val_classes} of {NUMBER_OF_CLASSES} total classes')

# Seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# Function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    # Convert image to floats in [0, 1] range
    #image = tf.cast(image, tf.float32) #/ 255.0
    # Explicit size needed for TPU
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

# This function parse our images and also get the target variable
def read_tfrecord(example):
    TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "target": tf.io.FixedLenFeature([], tf.int64)
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
    target = tf.cast(example['target'], tf.int32)
    return image, target

# This function load our tf records and parse our data with the previous function
def load_dataset(filenames, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway

    ignore_order = tf.data.Options()
    if not ordered:
        # Disable order, increase speed
        ignore_order.experimental_deterministic = False 
    
    # Automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # Use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # Returns a dataset of (image, label) pairs
    dataset = dataset.map(read_tfrecord, num_parallel_calls = AUTO) 
    return dataset

# This function output the data so that we can use arcface
def arcface_format(image, target):
    image = tf.cast(image, tf.float32) / 255.0
    return {'inp1': image, 'inp2': target}, target
# augment

def augment(image, target):
    image = tf.cast(image, tf.uint8)
    image = tf.image.random_flip_left_right(image)
    image = tf.cast(image, tf.float32)
    return image, target


# Training data pipeline
def get_training_dataset(filenames, ordered = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(augment, num_parallel_calls = AUTO)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    # The training dataset must repeat for several epochs
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # Prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Validation data pipeline
def get_validation_dataset(filenames, ordered = True, prediction = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    # If we are in prediction mode, use bigger batch size for faster prediction
    if prediction:
        dataset = dataset.batch(BATCH_SIZE * 4)
    else:
        dataset = dataset.batch(BATCH_SIZE)
    # Prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

# Count the number of observations with the tabular csv
def count_data_items(filenames):
    records = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    df = pd.read_csv(DICT_PATH)
    n = df[df['group'].isin(records)].shape[0]
    return n

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES  = count_data_items(VALIDATION_FILENAMES)
print(f'Training with {NUM_TRAINING_IMAGES} images')
print(f'Validating with {NUM_VALIDATION_IMAGES} images')


tf.keras.utils.get_custom_objects()['ArcMarginProduct'] = ArcMarginProduct
# Function to build our model using fine tunning (efficientnet)
def get_model():
    with strategy.scope():

        margin = ArcMarginProduct(
            n_classes = NUMBER_OF_CLASSES, 
            s = 64, 
            m = 0.2, 
            name='head/arc_margin', 
            dtype='float32'
            )

        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x = efn.EfficientNetB5(weights = 'imagenet', include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        x = tf.keras.layers.Dense(2048)(x)
        #x = tf.keras.layers.Dense(2048)(x)
        #x = tf.keras.layers.Dense(2048)(x)
        x = margin([x, label])
    
        output = tf.keras.layers.Softmax(dtype='float32')(x)
        model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
        opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)

        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
    
        return model

# Seed everything
seed_everything(SEED)

# Build training and validation generators
train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered = False)
val_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True, prediction = False)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
filepath = f'b5.h5'
# Using a checkpoint to save best model (want the entire model, not only the weights)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                        monitor = 'val_loss', 
                        save_best_only = True, 
                        save_weights_only = False)
# Using learning rate scheduler
cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', 
                            mode = 'min', 
                            factor = 0.5, 
                            patience = 1, 
                            verbose = 1, 
                            min_delta = 0.0001)


model = get_model()
# Train and evaluate our model
history = model.fit(train_dataset,  
            steps_per_epoch = STEPS_PER_EPOCH,
            epochs = EPOCHS,
            callbacks = [cb_lr_schedule, checkpoint],
            validation_data = val_dataset,
            verbose = 2)


## Inference推断阶段
* 构建测试集与训练集图片嵌入向量索引
* 相似图片排序
* 返回最高分图片标签
* 输出标签与得分

In [ ]:
import operator
import gc
import pathlib
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from scipy import spatial
import cv2
!pip install ../input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/efficientnetrepo110/efficientnet-1.1.0-py3-none-any.whl
import efficientnet.tfkeras as efn
import math

NUMBER_OF_CLASSES = 81313
IMAGE_SIZE = [384, 384]
LR = 0.0001

class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output


# Function to build our model using fine tunning (efficientnet)
def get_model():

    margin = ArcMarginProduct(
        n_classes = NUMBER_OF_CLASSES, 
        s = 30, 
        m = 0.5, 
        name='head/arc_margin', 
        dtype='float32'
        )

    inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    x0 = efn.EfficientNetB5(weights = None, include_top = False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x0)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(2048)(x) #-4
    x = margin([x, label])

    output = tf.keras.layers.Softmax(dtype='float32')(x)
    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    opt = tf.keras.optimizers.Adam(learning_rate = LR)

    model.compile(
        optimizer = opt,
        loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
        ) 

    return model


NUM_EMBEDDING_DIMENSIONS = 2048
DATASET_DIR = '../input/landmark-image-train/train_encoded.csv'
TEST_IMAGE_DIR = '../input/landmark-recognition-2021/test'
TRAIN_IMAGE_DIR = '../input/landmark-recognition-2021/train'
MODEL = get_model()

MODEL = tf.keras.models.Model(inputs = MODEL.input[0], outputs = MODEL.layers[-4].output)
MODEL.load_weights('b5.h5')
NUM_TO_RERANK = 3


NUM_PUBLIC_TEST_IMAGES = 10345 # Used to detect if in session or re-run.

# Read image and resize it
def read_image(image_path, size = (384, 384)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tostring()
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.reshape(img, [1, 384, 384, 3])
    return img

# Function to get training and test embeddings
def generate_embeddings(filepaths):
    image_paths = [x for x in pathlib.Path(filepaths).rglob('*.jpg')]
    num_images = len(image_paths)
    ids = num_images * [None]
    # Generate an empty matrix where we can store the embeddings of each image
    embeddings = np.empty((num_images, NUM_EMBEDDING_DIMENSIONS))
    for i, image_path in enumerate(image_paths):
        ids[i] = image_path.name.split('.')[0]
        image_tensor = read_image(str(image_path), (384, 384))
        prediction = MODEL.predict(image_tensor) # 2048d
        embeddings[i, :] = tf.nn.l2_normalize(prediction, axis=1, epsilon=1e-12, name=None)#prediction
    return ids, embeddings

# This function get the most similar train images for each test image based on cosine similarity
def get_similarities(train_csv, test_directory, train_directory):
    # Get target dictionary
    df = pd.read_csv(train_csv)
    df = df[['id', 'landmark_id']]
    df.set_index('id', inplace = True)
    df = df.to_dict()['landmark_id']
    # Extract the test ids and global feature for the test images
    test_ids, test_embeddings = generate_embeddings(test_directory) #test 
    # Extract the train ids and global features for the train images
    train_ids, train_embeddings = generate_embeddings(train_directory) 
    # Initiate a list were we will store the similar training images for each test image (also score)
    train_ids_labels_and_scores = [None] * test_embeddings.shape[0]
    # Using (slow) for-loop, as distance matrix doesn't fit in memory
    for test_index in range(test_embeddings.shape[0]):
        distances = spatial.distance.cdist(
            test_embeddings[np.newaxis, test_index, : ], train_embeddings, 'cosine')[0]
        # Get the indices of the closest images
        top_k = np.argpartition(distances, NUM_TO_RERANK)[:NUM_TO_RERANK]
        # Get the nearest ids and distances using the previous indices
        nearest = sorted([(train_ids[p], distances[p]) for p in top_k], key = lambda x: x[1])
        # Get the labels and score results
        train_ids_labels_and_scores[test_index] = [(df[train_id], 1.0 - cosine_distance) for \
                                                   train_id, cosine_distance in nearest]
        
    del test_embeddings
    del train_embeddings
    gc.collect()
    return test_ids, train_ids_labels_and_scores

# This function aggregate top simlarities and make predictions
def generate_predictions(test_ids, train_ids_labels_and_scores):
    targets = []
    scores = []
    
    # Iterate through each test id
    for test_index, test_id in enumerate(test_ids):
        aggregate_scores = {}
        # Iterate through the similar images with their corresponing score for the given test image
        for target, score in train_ids_labels_and_scores[test_index]:
            if target not in aggregate_scores:
                aggregate_scores[target] = 0
            aggregate_scores[target] += score
        # Get the best score
        target, score = max(aggregate_scores.items(), key = operator.itemgetter(1))
        targets.append(target)
        scores.append(score)
        
    final = pd.DataFrame({'id': test_ids, 'target': targets, 'scores': scores})
    final['landmarks'] = final['target'].astype(str) + ' ' + final['scores'].astype(str)
    final[['id', 'landmarks']].to_csv('submission.csv', index = False)
    return final

def inference_and_save_submission_csv(train_csv, test_directory, train_directory):
    image_paths = [x for x in pathlib.Path(test_directory).rglob('*.jpg')]
    test_len = len(image_paths)
    if test_len == NUM_PUBLIC_TEST_IMAGES:
        # Dummy submission
        shutil.copyfile('../input/landmark-recognition-2021/sample_submission.csv', 'submission.csv')
        return 'Job Done'
    else:
        test_ids, train_ids_labels_and_scores = get_similarities(train_csv, test_directory, train_directory)
        final = generate_predictions(test_ids, train_ids_labels_and_scores)
        return final
    
final = inference_and_save_submission_csv(DATASET_DIR, TEST_IMAGE_DIR, TRAIN_IMAGE_DIR)

# 总结
本解决方案基于度量学习来对众多地标类别进行分类，分别利用Efficientnet-b7与Efficientnet-b6在imagenet的预训练权重，在超大地标图像数据集重新训练，进一步在较大尺寸图像上微调，调整ArcFace的超平面参数，有效获取图像嵌入空间。在得到指定图像库与待识别图像库嵌入向量后，逐一计算两个图库图像余弦距离，利用余弦距离进一步重新排序获取最高者的标签索引与分数，生成最终结果。